In [1]:
#!/usr/bin/env python

import pandas as pd
import subprocess
import sys
import os

def mergelinear(tissue, PATH, permuted=False):
    perm=''
    if permuted: perm='_perm'
    print ('\n',tissue , '... ',perm )
    Frames=[pd.read_csv(PATH+ tissue+'/chr1/Lin_Reg_Out'+perm,sep='\t')]
    CHROM = [x for x in range(2,23)] +['X','Y']
    for x in CHROM:
        LN=PATH+ tissue +'/chr'+str(x)+'/Lin_Reg_Out'+perm
        frame1=pd.read_csv(LN, sep='\t')
        Frames.append(frame1)
    Results=pd.concat(Frames)
    print ('All Chrms ','   ', Results.shape[0],'tests saved in ', PATH+ tissue )
    Results.to_csv(PATH + tissue +'/Lin_Reg_Out'+perm, sep='\t', header=True, index=False)
    return(0)

def mergequadratic(tissue, PATH, permuted=False):
    perm=''
    if permuted: perm='_perm'
    print (tissue , '... START')
    Frames=[pd.read_csv(PATH+ tissue+'/QuadraticReg/quad_regression'+perm+'_chr1.tab',sep='\t')]
    CHROM = [x for x in range(2,23)] +['X','Y']
    for x in CHROM:
        Reg = PATH+tissue+'/QuadraticReg/quad_regression'+perm+'_chr'+str(x)+'.tab'
        frame1=pd.read_csv(Reg, sep='\t')
        Frames.append(frame1)
    Results=pd.concat(Frames)
    print ('\n All Chrms ','   ', Results.shape)
    Results.to_csv(PATH+ tissue +'/QuadraticReg/quadratic_reg_out'+perm+'.table', sep='\t', header=True, index=False)
    return(0)

"""
    Merge Reregression files into one single file 
    
"""
SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral":"Adipose V.",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "Skin Unexposed",
    "Skin-SunExposed":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}
TISSUES = sorted(SHORTEN.keys())
PATH = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'


In [2]:
#Merge regression
for tissue in TISSUES:
    mergelinear(tissue, PATH, permuted=False)
    mergelinear(tissue, PATH, permuted=True)
       


 Adipose-Subcutaneous ...  
All Chrms      281318 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Adipose-Subcutaneous

 Adipose-Subcutaneous ...  _perm
All Chrms      281318 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Adipose-Subcutaneous

 Adipose-Visceral ...  
All Chrms      281223 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Adipose-Visceral

 Adipose-Visceral ...  _perm
All Chrms      281223 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Adipose-Visceral

 Artery-Aorta ...  
All Chrms      277044 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Artery-Aorta

 Artery-Aorta ...  _perm
All Chrms      277044 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Artery-Aorta

 Artery-Tibial ...  
All Chrms      275588 tests saved in  /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Artery-Tibial

 Artery-Tibial ...  _perm
All Chrms      275588 test

In [18]:
#Merge ANOVA results
head='echo "chrom,gene,str.start,numsnps,numsamples,r2_str,r2_snp,r2_snpstr,anova_pval,estr_fdr,esnp_fdr,delta_bic,delta_aic" >'+PATH
ch = [str(x) for x in range(1,23,1)]+['X']

for T in TISSUES[1:]:
    
    frames=[]
    
    #print(T, ' ',PATH+T+"/HH/anova_ch"+x )
    
    for x in ch:
        command='grep "^chr" '+PATH+T+"/HH/anova_ch"+x+" > tmp"
        #print(command)
        cmd = os.system(command)
        C = pd.read_csv('tmp')
        
        if 'number_top_snp' in C.columns:
            del C['number_top_snp']
        frames.append(C)
        #print(command)
        cmd=os.system(command)
        
    pd.concat(frames).to_csv(PATH+T+"/HH/anova_wg.csv", sep='\t', index=None)
    ANO = pd.read_csv(PATH+T+"/HH/anova_wg.csv",sep='\t')
    QVL = pd.read_csv(PATH+T+"/PQValues", sep='\t')
    print (ANO.shape[0] ,'anova', QVL.loc[QVL['qvalue']<0.1].shape, T)
    break

1751 anova (1842, 11) Adipose-Visceral


In [19]:
#ANO = pd.read_csv(PATH+T+"/HH/anova_wg.csv",sep='\t')
Y =[x for x in set(list(QVL.loc[QVL['qvalue']<0.1]['gene'])) if x not in set(list(ANO['gene']))]
print(Y)
len(Y)

PROGRESS("ANOVA was restricted to %s genes.... %s genes were not filtered out (may be eSTRs) and will be analyzed."%(str(len(exclusive_genes_list)), str(len(allgenes))) )     

['ENSG00000239713.3', 'ENSG00000137411.12', 'ENSG00000183888.4', 'ENSG00000157326.14', 'ENSG00000102753.5', 'ENSG00000124486.8', 'ENSG00000164366.3', 'ENSG00000109466.9', 'ENSG00000105136.15', 'ENSG00000229676.2', 'ENSG00000223496.1', 'ENSG00000197465.9', 'ENSG00000166321.9', 'ENSG00000108578.10', 'ENSG00000146707.10', 'ENSG00000198231.8', 'ENSG00000075407.13', 'ENSG00000225190.4', 'ENSG00000150316.7', 'ENSG00000221829.5', 'ENSG00000128487.12', 'ENSG00000205420.6', 'ENSG00000196199.9', 'ENSG00000168961.12', 'ENSG00000184227.3', 'ENSG00000178980.10', 'ENSG00000204516.5', 'ENSG00000196268.7', 'ENSG00000120328.4', 'ENSG00000087191.8', 'ENSG00000138735.11', 'ENSG00000156968.8', 'ENSG00000204520.8', 'ENSG00000152439.8', 'ENSG00000254999.2', 'ENSG00000087842.6', 'ENSG00000204577.7', 'ENSG00000188659.5', 'ENSG00000100197.16', 'ENSG00000184083.7', 'ENSG00000142794.14', 'ENSG00000140876.7', 'ENSG00000095917.9', 'ENSG00000121716.14', 'ENSG00000122223.8', 'ENSG00000197915.5', 'ENSG00000183628.8',

94